 The code in this notebook, **when run in order**, will:

#### (1) Produce a dataframe of tweets for a certain keyword (columns right now are, location and tweet text; can modify the search criteria as necessary (location lists, hashtags, etc.), saved as a csv

#### (2) Delete the duplicates/very similar spam tweets but leave one copy behind. 

#### (3) Clean the remaining tweets and saves as a csv

By the end of the notebook, you'll have a clean twitter dataframe that's ready for analysis 

In [1]:
import os
import tweepy
import pandas as pd
import time
import itertools
import numpy as np
from itertools import chain


In [2]:
consumer_key= '3emnsuDpqLBXlxD5UPUxVnxTt'
consumer_secret= 'ZPZBilbjbca7pYAmMv05hlNKsxLb4CJd6A5kNa2JNIs62BjF7m'
access_token= '1406213154-M77vYKXz7wIAUzzJkqLVzfDH0D6K11xMJkzGHzJ'
access_token_secret= 'iLcP4BuW4jWRYIHEiIZwsD5jgYlMY1aTmtSFwPQqbaQrX'

In [3]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

## Specify tweet search/filtering criteria here:


In [4]:
search_words_free = "\"mask free\" OR #maskfree OR maskfree"
search_words_up = "\"mask up\" OR #maskup OR maskup"
date_since = "2021-03-10"
new_search_free = search_words_free + " -filter:retweets"
new_search_up = search_words_up + " -filter:retweets"

## Set location labels and corresponding location coordinates and radii here:

In [5]:
## ** In the order of the counties listed in county_info_new.csv

locs_labels = [
'New York County NY',
 'County of San Francisco CA',
 'Hudson County NJ',
 'Suffolk County MA',
 'Philadelphia County PA',
 'Washington DC',
 'Alexandria VA',
 'Baltimore MD',
 'Cook County IL',
 'St. Louis County MO',
 'Milwaukee County WI',
 'Denver County CO',
 'Orange County CA',
 'Pinellas County FL',
 'Ramsey County MN',
 'Wayne County, MI',
 'Cuyahoga County OH',
 'Dallas County TX',
 'DeKalb County GA',
 'Salt Lake County UT',
 'Marion County IN',
 'Jefferson County KY',
 'Johnson County KS',
 'King County WA',
 'Austin TX']


locs = [
    '40.7741618671,-73.9697971209,100mi',
    '37.7597150131,-122.693975573,100mi',
    '40.7309049975,-74.0759554422,100mi',
    '42.3486893925,-70.9856096697,100mi',
    '40.0076360466,-75.1339446794,100mi', 
    '38.9047737146,-77.0163026702,100mi',
    '38.8184535854,-77.0862060584,100mi',
    '39.443072852,-76.6163229502,100mi',
    '41.8954290221,-87.6461407931,100mi',
    '38.6406885662,-90.4433975603,100mi',
    '43.0154568883,-87.5805486607,100mi',
    '39.7621646287,-104.875801423,100mi',
    '33.6769134328,-117.776142822,100mi',
    '27.9026762041,-82.7395539464,100mi',
    '45.0170541432,-93.099606274,100mi',
    '42.2847454761,-83.2611328686,100mi',
    '41.6350590585,-81.7001959208,100mi',
    '32.7666300195,-96.7778770663,100mi',
    '33.7715468713,-84.226432416,100mi',
    '40.6673139262,-111.923601307,100mi',
    '39.7817096805,-86.1384716709,100mi',
    '38.1871910002,-85.6591577058,100mi',
    '38.8837685125,-94.8222610425,100mi',
    '47.4909231926,-121.83595173,100mi',
    '29.88548725,-96.27736949,100mi']

## Just zip locs_labels with locs and you'd have the coordinates mapped to their names

mapped_locations = list(zip(locs_labels, locs))

In [6]:
mapped_locations

[('New York County NY', '40.7741618671,-73.9697971209,100mi'),
 ('County of San Francisco CA', '37.7597150131,-122.693975573,100mi'),
 ('Hudson County NJ', '40.7309049975,-74.0759554422,100mi'),
 ('Suffolk County MA', '42.3486893925,-70.9856096697,100mi'),
 ('Philadelphia County PA', '40.0076360466,-75.1339446794,100mi'),
 ('Washington DC', '38.9047737146,-77.0163026702,100mi'),
 ('Alexandria VA', '38.8184535854,-77.0862060584,100mi'),
 ('Baltimore MD', '39.443072852,-76.6163229502,100mi'),
 ('Cook County IL', '41.8954290221,-87.6461407931,100mi'),
 ('St. Louis County MO', '38.6406885662,-90.4433975603,100mi'),
 ('Milwaukee County WI', '43.0154568883,-87.5805486607,100mi'),
 ('Denver County CO', '39.7621646287,-104.875801423,100mi'),
 ('Orange County CA', '33.6769134328,-117.776142822,100mi'),
 ('Pinellas County FL', '27.9026762041,-82.7395539464,100mi'),
 ('Ramsey County MN', '45.0170541432,-93.099606274,100mi'),
 ('Wayne County, MI', '42.2847454761,-83.2611328686,100mi'),
 ('Cuyahoga

In [7]:
len(locs) # ensure that locs and loc_labels lens match; should have through zip funct anyway

25

In [8]:
len(locs_labels)

25

In [9]:
# SCRAPING TWEETS FOR ALL LOCATIONS # #Kaylah added date info here
def gather_tweets(search, date_since, geocodes):
    tweets_list = []
    for loc in geocodes:
        try:
            tweets = tweepy.Cursor(api.search,q=search, lang="en", since=date_since, geocode=loc[1], tweet_mode='extended').items(1000) 
            tweets_list.append((loc[0], [[tweet.full_text, tweet.created_at.strftime("%d-%b-%Y")] for tweet in tweets]))
            tweets_df = pd.DataFrame(tweets_list)

        except BaseException as e:
            print('failed on_status,',str(e))
            time.sleep(3)
    return tweets_df


In [12]:
mask_free_df = gather_tweets(search_words_free, date_since, mapped_locations)

mask_free_df.head()

TypeError: gather_tweets() takes 1 positional argument but 3 were given

In [ ]:
mask_up_df = gather_tweets(search_words_up, date_since, mapped_locations)

mask_up_df.head()

[('New York County NY', [["RT @MountSinaiNYC: Mask wearing saves lives. Keep doing what's right. #MaskUp and stay safe. Learn more: https://t.co/79N9QY4F7t #COVID19 h…", '19-Mar-2021'], ['@timcorrimal @ExposingALEC @Progress2day @followthewabbit @TheRReport @SigHolmes @aimeehogan8 @artonissues Stay safe...still Mask up ...double mask 😷😷....and yes get vaccinated 😷💙🙏😷💙🙏', '19-Mar-2021'], ['RT @SteveBellone: Today, there were 711 new cases of COVID-19 reported in Suffolk, for a positivity rate of 3.9%. \n\nWe are working around t…', '19-Mar-2021'], ["RT @DrTomFrieden: I'm worried about the Covid spikes we're seeing in Europe. We can avoid a similar scenario in the US if we ramp up vaccin…", '19-Mar-2021'], ["RT @DrTomFrieden: I'm worried about the Covid spikes we're seeing in Europe. We can avoid a similar scenario in the US if we ramp up vaccin…", '19-Mar-2021'], ['RT @DrTomFrieden: Five points to end the week.\n\nFirst, Covid is far from over. Have a look at the map from @NYTimes below

failed on_status, 
failed on_status, 
failed on_status, 
failed on_status, 


In [ ]:
# saves the dataframe if desired
'''
mask_free_df.to_csv('mask_free_dataset.csv') 
mask_up_df.to_csv('mask_up_dataset.csv')
'''

################################################################################################

# Tweet df loading and cleaning

In [11]:
import os
import tweepy
import pandas as pd
import time
import itertools
import numpy as np
from itertools import chain
import regex as re
import unicodedata
import nltk

## Loads tweet dataframe, ensures datatypes are appropriate, flattens tweets into one list for ease, labels columns


In [ ]:
#mask_free_df = pd.read_csv('mask_free_dataset.csv', converters={'1': eval}, index_col=0)
mask_free_df['tweets_list'] = mask_free_df['1'].apply(lambda x : np.array(x).flatten())
mask_free_df.columns = ['county', 'original_tweet', 'tweets_list']

mask_free_df.loc[0, 'county'] = 'Austin County TX' #reformatting test case
mask_free_df


In [ ]:
#mask_up_df = pd.read_csv('mask_up_dataset.csv', converters={'1': eval}, index_col=0)
mask_up_df['tweets_list'] = mask_up_df['1'].apply(lambda x : np.array(x).flatten())
mask_up_df.columns = ['county', 'original_tweet', 'tweets_list']

mask_up_df.loc[0, 'county'] = 'Austin County TX' #reformatting test case
mask_up_df

## Now, expands dataset so there is one row per tweet (before, all tweets for a county were in one row)


In [13]:
free_explode = mask_free_df.explode('original_tweet')
free_explode

KeyError: 'original_tweet'

In [ ]:
up_explode = mask_up_df.explode('original_tweet')
up_explode

In [ ]:
# Creating new column for date tweeted

def add_date(df):
    df['date_tweeted'] = '0'

    for tweet_ind, tweet_row, in df.iterrows():
        tweet_row['date_tweeted'] = tweet_row['original_tweet'][1]

    # cleaning date layout for easier matching to COVID-19 dataframe

    for date_ind, date_row in df.iterrows():
        date_alt = date_row['date_tweeted'].replace('-', ' ')
        date_alt = date_alt.split()
        date_alt[1] = '3'
        date_alt[2] = '21'
        date_alt[0], date_alt[1] = date_alt[1], date_alt[0]
        date_alt = "/".join(date_alt)
        date_row['date_tweeted'] = date_alt
        
    return df

In [ ]:
free_explode = add_date(free_explode)
up_explode = add_date(up_explode)

In [ ]:
free_explode['original_tweet'].iloc[56] # test output

***********
# See note:

I tried getting rid of duplicates a few ways; doing pairwise levenshtein distance comparisons between all the tweets took way too long, and creating a mask to then drop duplicate tweets was also complicated. I noticed that most very-similar-basically-duplicate tweets had the same text, but different handles and urls. So I took a slice of part of the text that was very likely to be shared between the similar tweets, created a new column in the dataset where each tweet-row got a corresponding tweet-slice, and then I deleted the rows where there were duplicate tweet-slices. As you can see, this resulted in a *lot* of data loss (3000+ tweets down to 850; see twitter_drop a few cells down), because many of them were duplicate/spam tweets.
***********

In [ ]:
# converting to string data type 
free_explode["tweet_str"]= free_explode["original_tweet"].astype(str)
up_explode["tweet_str"]= up_explode["original_tweet"].astype(str)
  
# slicing till 2nd last element 
free_explode["tweet_slice"] = free_explode["tweet_str"].str.slice(50,1000)
up_explode["tweet_slice"] = up_explode["tweet_str"].str.slice(50,1000) 

  
# display 
free_explode
up_explode

### Can look through slice list to ensure that the slices look right

In [ ]:
slice_list = free_explode['tweet_slice'].tolist()
for i, v in enumerate(slice_list):
    print(i,v)
#slice_list[184]

slice_list = up_explode['tweet_slice'].tolist()
for i, v in enumerate(slice_list):
    print(i,v)
#slice_list[184]

In [ ]:
free_drop = free_explode.drop_duplicates(subset=['tweet_slice'], keep='last')
free_drop

In [ ]:
up_drop = up_explode.drop_duplicates(subset=['tweet_slice'], keep='last')
up_drop

In [ ]:
free_drop['tweet_str'].iloc[89] #test

In [ ]:
free_drop['tweet_str'].iloc[88] #test

## Cleans the tweets


In [ ]:
free_df_clean = free_drop
up_df_clean = up_drop

In [ ]:
def clean_tweets(df_clean):
    # lower case
    df_clean['tweet_text'] = df_clean['tweet_str'].str.lower()

    # remove url links
    df_clean['tweet_text'] = df_clean['tweet_text'].apply(lambda x: re.sub(r'https?:\/\/\S+', '', x))

    # remove url/website that didn't use http, is only checking for .com websites 
    # so words that are seperated by a . are not removed
    df_clean['tweet_text'] = df_clean['tweet_text'].apply(lambda x: re.sub(r"www\.[a-z]?\.?(com)+|[a-z]+\.(com)", '', x))
    # remove @mention
    df_clean['tweet_text'] = df_clean['tweet_text'].apply(lambda x: re.sub(r'@mention', '', x))
    # remove {link}
    df_clean['tweet_text'] = df_clean['tweet_text'].apply(lambda x: re.sub(r'{link}', '', x))
    # remove &text; html chars
    df_clean['tweet_text'] = df_clean['tweet_text'].apply(lambda x: re.sub(r'&[a-z]+;', '', x))
    # [video]
    df_clean['tweet_text'] = df_clean['tweet_text'].apply(lambda x: re.sub(r"\[video\]", '', x))
    # remove all remaining characters that aren't letters, white space, or 
    # the following #:)(/\='] that are used in emojis or hashtags
    df_clean['tweet_text'] = df_clean['tweet_text'].apply(lambda x: re.sub(r"[^a-z\s\(\-:\)\\\/\];='#]", '', x))
    return df_clean

In [ ]:
free_df_clean = clean_tweets(free_df_clean)
free_df_clean

In [ ]:
up_df_clean = clean_tweets(up_df_clean)
up_df_clean

## Do a spot check to make sure that the data has been cleaned well:


In [ ]:
free_df_clean['tweet_text'].iloc[5]

In [ ]:
free_drop['tweet_str'].iloc[5]

# Save as csv if desired

In [ ]:
'''
free_df_clean.to_csv('cleaned_maskfree_tweets.csv')
up_df_clean.to_csv('cleaned_maskup_tweets.csv')
'''

# Merging cleaned tweets dataframe with county and COVID-19 data

## First, merging county info with COVID case data

### Loading time-series data before joining


In [14]:
import pandas as pd

c19cases = pd.read_csv('time_series_covid19_confirmed_US.csv')

c19cases.head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,3/9/21,3/10/21,3/11/21,3/12/21,3/13/21,3/14/21,3/15/21,3/16/21,3/17/21,3/18/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,6371,6400,6409,6409,6416,6426,6471,6474,6483,6495
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,19962,20012,20044,20072,20091,20103,20210,20227,20263,20287
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,2147,2161,2171,2175,2181,2184,2195,2198,2199,2202
3,84001007,US,USA,840,1007.0,Bibb,Alabama,US,32.996421,-87.125115,...,2466,2469,2474,2475,2479,2481,2499,2508,2512,2519
4,84001009,US,USA,840,1009.0,Blount,Alabama,US,33.982109,-86.567906,...,6255,6260,6274,6282,6288,6291,6353,6361,6371,6376


### Loading selected county information

In [15]:
# dataframe of selected counties
counties = pd.read_csv('county_info_new.csv', index_col = [0]) #note csv file name
counties.rename({'counties': 'county'}, axis='columns', inplace = True)
counties

,county,state,populations,ed_rate,tier,lat/lon,FIPS County Code
0,New York,New York,69468,0.613,2,"40.7741618671, -73.9697971209",36061
1,San Francisco,California,17179,0.581,2,"37.7597150131, -122.693975573",6075
2,Hudson,New Jersey,13732,0.423,1,"40.7309049975, -74.0759554422",34017
3,Suffolk,Massachusetts,12417,0.461,2,"42.3486893925, -70.9856096697",36103
4,Philadelphia,Pennsylvania,11379,0.297,1,"40.0076360466, -75.1339446794",42101
5,District of Columbia,District of Columbia,9857,0.585,2,"38.9047737146, -77.0163026702",11001
6,Alexandria,Virginia,9314,0.631,3,"38.8184535854, -77.0862060584",51510
7,Baltimore,Maryland,7672,0.394,1,"39.443072852, -76.6163229502",24005
8,Cook,Illinois,5495,0.388,1,"41.8954290221, -87.6461407931",17031
9,St. Louis,Missouri,5157,0.436,1,"38.6406885662, -90.4433975603",29189


In [16]:
# Collecting COVID-19 data according to the counties listed in county_info_new.csv

c19df = pd.DataFrame()

for index, county in counties.iterrows():
    c19df = c19df.append(c19cases[(c19cases['Province_State'] == str(county['state'])) & 
                          (c19cases['Admin2'] == str(county['county']))])
    
    

c19df

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,3/9/21,3/10/21,3/11/21,3/12/21,3/13/21,3/14/21,3/15/21,3/16/21,3/17/21,3/18/21
1930,84036061,US,USA,840,36061.0,New York,New York,US,40.767273,-73.971526,...,108580,109015,109618,110289,110727,111336,111991,112440,112911,113144
233,84006075,US,USA,840,6075.0,San Francisco,California,US,37.752151,-122.438567,...,34634,34654,34688,34722,34758,34784,34810,34820,34820,34820
1850,84034017,US,USA,840,34017.0,Hudson,New Jersey,US,40.737662,-74.075157,...,71969,72300,72644,73009,73292,73595,73816,74064,74437,74909
1277,84025025,US,USA,840,25025.0,Suffolk,Massachusetts,US,42.327951,-71.078504,...,79835,80026,80231,80450,80655,80886,81002,81092,81293,81519
2380,84042101,US,USA,840,42101.0,Philadelphia,Pennsylvania,US,40.003385,-75.137927,...,120966,121242,121583,121881,121881,121881,122598,122971,123531,123959
337,84011001,US,USA,840,11001.0,District of Columbia,District of Columbia,US,38.904178,-77.016560,...,41910,42006,42128,42282,42432,42511,42623,42730,42811,42892
3010,84051510,US,USA,840,51510.0,Alexandria,Virginia,US,38.814003,-77.081831,...,10468,10490,10504,10529,10548,10570,10592,10617,10643,10671
1239,84024005,US,USA,840,24005.0,Baltimore,Maryland,US,39.457847,-76.629120,...,51966,52096,52271,52450,52606,52740,52828,52936,53073,53282
640,84017031,US,USA,840,17031.0,Cook,Illinois,US,41.841448,-87.816588,...,479913,480487,481101,481787,482424,482973,483282,484146,484719,485771
1642,84029189,US,USA,840,29189.0,St. Louis,Missouri,US,38.637700,-90.445837,...,89599,89767,89929,90113,90287,90398,90479,90641,90819,90956


In [17]:
#resetting index for ease of adding columns
c19newdf = pd.DataFrame(c19df)

c19newdf.reset_index()

,index,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,...,3/9/21,3/10/21,3/11/21,3/12/21,3/13/21,3/14/21,3/15/21,3/16/21,3/17/21,3/18/21
0,1930,84036061,US,USA,840,36061.0,New York,New York,US,40.767273,...,108580,109015,109618,110289,110727,111336,111991,112440,112911,113144
1,233,84006075,US,USA,840,6075.0,San Francisco,California,US,37.752151,...,34634,34654,34688,34722,34758,34784,34810,34820,34820,34820
2,1850,84034017,US,USA,840,34017.0,Hudson,New Jersey,US,40.737662,...,71969,72300,72644,73009,73292,73595,73816,74064,74437,74909
3,1277,84025025,US,USA,840,25025.0,Suffolk,Massachusetts,US,42.327951,...,79835,80026,80231,80450,80655,80886,81002,81092,81293,81519
4,2380,84042101,US,USA,840,42101.0,Philadelphia,Pennsylvania,US,40.003385,...,120966,121242,121583,121881,121881,121881,122598,122971,123531,123959
5,337,84011001,US,USA,840,11001.0,District of Columbia,District of Columbia,US,38.904178,...,41910,42006,42128,42282,42432,42511,42623,42730,42811,42892
6,3010,84051510,US,USA,840,51510.0,Alexandria,Virginia,US,38.814003,...,10468,10490,10504,10529,10548,10570,10592,10617,10643,10671
7,1239,84024005,US,USA,840,24005.0,Baltimore,Maryland,US,39.457847,...,51966,52096,52271,52450,52606,52740,52828,52936,53073,53282
8,640,84017031,US,USA,840,17031.0,Cook,Illinois,US,41.841448,...,479913,480487,481101,481787,482424,482973,483282,484146,484719,485771
9,1642,84029189,US,USA,840,29189.0,St. Louis,Missouri,US,38.637700,...,89599,89767,89929,90113,90287,90398,90479,90641,90819,90956


In [18]:
# Adding different naming conventions to dataframe to match with tweet collection dataframe. 
# THIS NEW LIST IS IN LINE WITH county_info_new.csv

tweet_locs = pd.Series([
'New York County NY',
 'County of San Francisco CA',
 'Hudson County NJ',
 'Suffolk County MA',
 'Philadelphia County PA',
 'Washington DC',
 'Alexandria VA',
 'Baltimore MD',
 'Cook County IL',
 'St. Louis County MO',
 'Milwaukee County WI',
 'Denver County CO',
 'Orange County CA',
 'Pinellas County FL',
 'Ramsey County MN',
 'Wayne County, MI',
 'Cuyahoga County OH',
 'Dallas County TX',
 'DeKalb County GA',
 'Salt Lake County UT',
 'Marion County IN',
 'Jefferson County KY',
 'Johnson County KS',
 'King County WA',
 'Austin County TX'])

c19newdf['county'] = tweet_locs.values

## Joining dataframes 

### Have to join the mask_free and mask_up dataframes first

#### Loading mask free first:

In [ ]:
#mask_free = pd.read_csv("cleaned_maskfree_tweets.csv", index_col=0)
mask_free = free_df_clean
mask_free['type'] = 'maskfree'
mask_free

#### Then mask up:

In [ ]:
#mask_up = pd.read_csv("cleaned_maskup_tweets.csv", index_col=0)
mask_up = up_df_clean
mask_up['type'] = 'maskup'
mask_up

In [ ]:
full_tweets = pd.concat([mask_free, mask_up], axis=0)

full_tweets

### We now have the full tweets data with a column for 'type' (mask free vs. mask on, so we can more easily get frequency counts). 

In [ ]:
county1 = full_tweets.county.unique()
county2 = c19newdf.county.unique()

# formatting individual dataframes for aid in visualizations later on
county1_free = mask_free.county.unique()
county1_up = mask_up.county.unique()

In [19]:
county1= set(county1)
county1_free = set(county1_free)
county1_up = set(county1_up)

county2 = set(county2)

NameError: name 'county1' is not defined

# Make sure this is true -- means that county data's counties and the tweet data's counties match

In [ ]:
set(county1) == set(county2)
set(county1_free) == set(county2)
set(county1_up) == set(county2)

## Now, merging the time series data with the full tweet data

In [ ]:
tweets_and_c19 = full_tweets.merge(c19newdf, on='county')
tweets_and_c19

In [ ]:
# again creating separate sets for data visualizations
free_tweets_and_c19 = mask_free.merge(c19newdf, on='county')
up_tweets_and_c19 = mask_up.merge(c19newdf, on='county')

In [20]:
# Dates from COVID-19 dataset that need to be dropped
dates = '1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20,4/22/20,4/23/20,4/24/20,4/25/20,4/26/20,4/27/20,4/28/20,4/29/20,4/30/20,5/1/20,5/2/20,5/3/20,5/4/20,5/5/20,5/6/20,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20,5/17/20,5/18/20,5/19/20,5/20/20,5/21/20,5/22/20,5/23/20,5/24/20,5/25/20,5/26/20,5/27/20,5/28/20,5/29/20,5/30/20,5/31/20,6/1/20,6/2/20,6/3/20,6/4/20,6/5/20,6/6/20,6/7/20,6/8/20,6/9/20,6/10/20,6/11/20,6/12/20,6/13/20,6/14/20,6/15/20,6/16/20,6/17/20,6/18/20,6/19/20,6/20/20,6/21/20,6/22/20,6/23/20,6/24/20,6/25/20,6/26/20,6/27/20,6/28/20,6/29/20,6/30/20,7/1/20,7/2/20,7/3/20,7/4/20,7/5/20,7/6/20,7/7/20,7/8/20,7/9/20,7/10/20,7/11/20,7/12/20,7/13/20,7/14/20,7/15/20,7/16/20,7/17/20,7/18/20,7/19/20,7/20/20,7/21/20,7/22/20,7/23/20,7/24/20,7/25/20,7/26/20,7/27/20,7/28/20,7/29/20,7/30/20,7/31/20,8/1/20,8/2/20,8/3/20,8/4/20,8/5/20,8/6/20,8/7/20,8/8/20,8/9/20,8/10/20,8/11/20,8/12/20,8/13/20,8/14/20,8/15/20,8/16/20,8/17/20,8/18/20,8/19/20,8/20/20,8/21/20,8/22/20,8/23/20,8/24/20,8/25/20,8/26/20,8/27/20,8/28/20,8/29/20,8/30/20,8/31/20,9/1/20,9/2/20,9/3/20,9/4/20,9/5/20,9/6/20,9/7/20,9/8/20,9/9/20,9/10/20,9/11/20,9/12/20,9/13/20,9/14/20,9/15/20,9/16/20,9/17/20,9/18/20,9/19/20,9/20/20,9/21/20,9/22/20,9/23/20,9/24/20,9/25/20,9/26/20,9/27/20,9/28/20,9/29/20,9/30/20,10/1/20,10/2/20,10/3/20,10/4/20,10/5/20,10/6/20,10/7/20,10/8/20,10/9/20,10/10/20,10/11/20,10/12/20,10/13/20,10/14/20,10/15/20,10/16/20,10/17/20,10/18/20,10/19/20,10/20/20,10/21/20,10/22/20,10/23/20,10/24/20,10/25/20,10/26/20,10/27/20,10/28/20,10/29/20,10/30/20,10/31/20,11/1/20,11/2/20,11/3/20,11/4/20,11/5/20,11/6/20,11/7/20,11/8/20,11/9/20,11/10/20,11/11/20,11/12/20,11/13/20,11/14/20,11/15/20,11/16/20,11/17/20,11/18/20,11/19/20,11/20/20,11/21/20,11/22/20,11/23/20,11/24/20,11/25/20,11/26/20,11/27/20,11/28/20,11/29/20,11/30/20,12/1/20,12/2/20,12/3/20,12/4/20,12/5/20,12/6/20,12/7/20,12/8/20,12/9/20,12/10/20,12/11/20,12/12/20,12/13/20,12/14/20,12/15/20,12/16/20,12/17/20,12/18/20,12/19/20,12/20/20,12/21/20,12/22/20,12/23/20,12/24/20,12/25/20,12/26/20,12/27/20,12/28/20,12/29/20,12/30/20,12/31/20,1/1/21,1/2/21,1/3/21,1/4/21,1/5/21,1/6/21,1/7/21,1/8/21,1/9/21,1/10/21,1/11/21,1/12/21,1/13/21,1/14/21,1/15/21,1/16/21,1/17/21,1/18/21,1/19/21,1/20/21,1/21/21,1/22/21,1/23/21,1/24/21,1/25/21,1/26/21,1/27/21,1/28/21,1/29/21,1/30/21,1/31/21,2/1/21,2/2/21,2/3/21,2/4/21,2/5/21,2/6/21,2/7/21,2/8/21,2/9/21,2/10/21,2/11/21,2/12/21,2/13/21,2/14/21,2/15/21,2/16/21,2/17/21,2/18/21,2/19/21,2/20/21,2/21/21,2/22/21,2/23/21,2/24/21,2/25/21,2/26/21,2/27/21,2/28/21,3/1/21,3/2/21,3/3/21,3/4/21,3/5/21,3/6/21,3/7/21,3/8/21,3/9/21'
    
dates = dates.split(",")

In [ ]:
# droping dates
tweets_and_c19 = tweets_and_c19.drop(columns = dates)

tweets_and_c19

In [ ]:
# drop dates for individual sets
free_tweets_and_c19 = free_tweets_and_c19.drop(columns = dates)
up_tweets_and_c19 = up_tweets_and_c19.drop(columns = dates)

In [ ]:
# Export Data to CSV if desired
'''
tweets_and_c19.to_csv('tweets_and_c19.csv')
free_tweets_and_c19.to_csv('maskfree_tweets_and_c19.csv')
up_tweets_and_c19.to_csv('maskup_tweets_and_c19.csv')
'''

## Reading in the mask compliance dataset


In [21]:
# Reading in the mask usage csv from the New York Times
maskusage = pd.read_csv('mask-use-by-county.csv')

In [22]:
maskusage.head()

,COUNTYFP,NEVER,RARELY,SOMETIMES,FREQUENTLY,ALWAYS
0,1001,0.053,0.074,0.134,0.295,0.444
1,1003,0.083,0.059,0.098,0.323,0.436
2,1005,0.067,0.121,0.120,0.201,0.491
3,1007,0.020,0.034,0.096,0.278,0.572
4,1009,0.053,0.114,0.180,0.194,0.459


In [23]:
# Renaming County Code column to match original dataframe
maskusage_new = maskusage.rename(columns={'COUNTYFP': 'FIPS'})

maskusage_new.head()

,FIPS,NEVER,RARELY,SOMETIMES,FREQUENTLY,ALWAYS
0,1001,0.053,0.074,0.134,0.295,0.444
1,1003,0.083,0.059,0.098,0.323,0.436
2,1005,0.067,0.121,0.120,0.201,0.491
3,1007,0.020,0.034,0.096,0.278,0.572
4,1009,0.053,0.114,0.180,0.194,0.459


# Merging with the tweet-and-timeseries data:

In [ ]:
# Merging the Mask Usage dataset and Twitter/COVID dataset
usage_tweet_covid = tweets_and_c19.merge(maskusage_new, on='FIPS')

usage_tweet_covid

In [ ]:
# Merging the individual Mask Usage dataset and Twitter/COVID dataset
usage_freetweet_covid = free_tweets_and_c19.merge(maskusage_new, on='FIPS')
usage_uptweet_covid = up_tweets_and_c19.merge(maskusage_new, on='FIPS')


In [ ]:
counties = counties.rename(columns={'FIPS County Code': 'FIPS'})
counties

In [ ]:
final_merge = usage_tweet_covid.merge(counties, on='FIPS')

final_merge.head()

In [ ]:
# repeating merge for individual dataframes
free_final_merge = usage_freetweet_covid.merge(counties, on='FIPS')
up_final_merge = usage_uptweet_covid.merge(counties, on='FIPS')


In [ ]:
# Export Final Datasets to CSV
final_merge.to_csv('usage_tweet_covid.csv')
free_final_merge.to_csv('usage_maskfree_covid.csv')
up_final_merge.to_csv('usage_maskup_covid.csv')